## <center> RecSys. Home Assignment 1</center>

Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке мы предлагаем вам потренироваться в этом, и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении рекомендательных моделей. 

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл. 
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл. 
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла. 
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **10 октября 23:59**. 

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла Name_Surname_HW1.ipynb.  

Удачи!

In [1]:
from typing import NamedTuple, Union
import tests
import torch
from decimal import Decimal
import numpy as np

C:\Users\dimaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )


def my_tests(cases, metric):
    flg = False
    for index, case in enumerate(cases):
        print(f"\ncase: {index}\n")
        for k in case['topk']:
            result  = metric(case["output"], case["target"], k)
            if result == case["expected"][k]:
                print(k, 'good')
            else:
                print(k, 'ALERT!')
                print(result)
                flg = True
                break
        if flg:
            break

# Precision

$$P@k = \frac{\sum_{i=1}^k [rel_{i}]}{k}$$

In [3]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    topk = min(target.shape[1], topk)
    result = np.mean(list(map(lambda x: x/topk, np.array(torch.sum(target_sorted_by_output[:, :topk], dim=1)))))
    return result

In [4]:
tests.run_precision(precision)

# Recall

$$P@k = \frac{\sum_{i=1}^k [rel_{i}]}{|rel_k|}$$

In [5]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    result = torch.div(torch.sum(target_sorted_by_output[:, :topk], dim=1), torch.sum(target_sorted_by_output, dim=1)).mean()
    if torch.isnan(result):
        result = 0.0
    return result

In [6]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP@k = \frac{\sum_{i=1}^{k} P@i \cdot [rel@ i]} {\sum_{i=1}^{k}[rel@i]}$$
$$MNAP@k = \frac{1}{N} \sum_{k=1}^{N} \frac{1}{min(k, m_u)} AP@k,$$

где $m_u$ - количество items с интеракциями у пользователя $u$ в тестовый период, <br>
$N$ - количество пользователей. 

In [7]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    return 0

In [8]:
tests.run_map(mnap)
tests.run_mnap(mnap)

AssertionError: Inputs:{
  "output": [
    [
      0.0,
      0.0,
      0.0,
      0.0
    ]
  ],
  "target": [
    [
      0.0,
      0.0,
      0.0,
      0.0
    ]
  ],
  "topk": 1
}
The (d)types do not match: <class 'int'> != <class 'float'>.

# Normalized Dicsounted Cumulative Gain


$$ NDCG @k = \frac{DCG@k}{IDCG@k},$$ где 
$$DCG@k = \sum_{i=1}^{k} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG@k = \sum_{i=1}^{|rel_{k}|} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$

In [ ]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    ideal_target = prepare_target(target, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_ndcg(ndcg)